In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import optuna

e:\Projects\ANN Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data
df = pd.read_csv('Churn_Modelling.csv')

# Prepossing The Data

In [3]:
# Drop unnecessary columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
# Encode categorical variables
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])
    
# One-hot encode Geography
geo_encoder = OneHotEncoder(sparse_output=False)
geo_encoded = geo_encoder.fit_transform(df['Geography'].values.reshape(-1, 1))
geo_df = pd.DataFrame(geo_encoded, columns=['France', 'Germany', 'Spain'])
df = pd.concat([df.drop('Geography', axis=1), geo_df], axis=1)

In [5]:
# Save The Encoded Data In Pickle File :
with open('lable_Encoding_gender.pkl','wb') as file:
    pickle.dump(le_gender,file)
    
with open('One_Hot_Encoding_Geography.pkl','wb') as file:
    pickle.dump(geo_encoder,file)    

In [6]:
# Split The Data Into Train & Test :
X=df.drop(['Exited'],axis=1)
y=df['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=50)
X_train.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
5189,826,1,41,5,146466.46,2,0,0,180934.67,0.0,0.0,1.0
7969,637,1,49,2,108204.52,1,1,0,169037.84,0.0,1.0,0.0
9039,545,0,44,1,0.00,2,1,1,82614.89,0.0,0.0,1.0
5208,779,0,38,7,0.00,2,1,1,138542.87,0.0,0.0,1.0
506,691,1,30,7,116927.89,1,1,0,21198.39,0.0,1.0,0.0


In [14]:
# Standardization (or z-score normalization) of the features data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save The StandardScaler Object In Pickle File :
with open('Feature_scaling.pkl','wb') as file:
    pickle.dump(scaler,file)

# Build Model Using Pytorch

In [11]:
# Model definition
class SalaryPredictor(nn.Module):
    def __init__(self, input_dim, nodes, layers, dropout_rate):
        super().__init__()
        
        # Input layer
        layers_list = [
            nn.Linear(input_dim, nodes),
            nn.BatchNorm1d(nodes),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        ]
        
        # Hidden layers
        current_nodes = nodes
        for _ in range(layers):
            next_nodes = max(current_nodes // 2, 64)
            layers_list.extend([
                nn.Linear(current_nodes, next_nodes),
                nn.BatchNorm1d(next_nodes),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            current_nodes = next_nodes
        
        # Output layer
        layers_list.append(nn.Linear(current_nodes, 1))
        
        self.model = nn.Sequential(*layers_list)
    
    def forward(self, x):
        return self.model(x)

# Create Optuna Object Function

In [17]:
# Optuna objective function
def objective(trial):
    # Hyperparameters to optimize
    layers = trial.suggest_int('layers', 1, 10)
    nodes = trial.suggest_categorical('nodes', [128, 256, 512])
    epochs = trial.suggest_int("epochs", 10, 50, step=10)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

    # Prepare data
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train.values)
    
    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Model, loss
    model = SalaryPredictor(X_train.shape[1], nodes, layers, dropout_rate)
    criterion = nn.BCEWithLogitsLoss()
    
    # Optimizer Selection
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Training Loop
    best_val_loss = float('inf')
    for _ in range(epochs):
        model.train()
        total_loss = 0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        if avg_loss < best_val_loss:
            best_val_loss = avg_loss
        
        if avg_loss < 1e-4:
            break
    
    # Validation
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.FloatTensor(X_test)
        y_val_tensor = torch.FloatTensor(y_test.values)
        val_predictions = model(X_val_tensor)
        val_loss = criterion(val_predictions, y_val_tensor.unsqueeze(1))
    
    return val_loss.item()

In [21]:
# Create Optuna study object and optimize it
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, show_progress_bar=True) 
# Increased trials for better optimization

[I 2025-01-25 15:15:08,607] A new study created in memory with name: no-name-44706f64-7a8d-411d-b9bd-07bcdf9b457a
  0%|          | 0/50 [00:00<?, ?it/s]C:\Users\mrray\AppData\Local\Temp\ipykernel_14452\4227408178.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
Best trial: 0. Best value: 0.42309:   2%|▏         | 1/50 [00:15<12:48, 15.68s/it]

[I 2025-01-25 15:15:24,289] Trial 0 finished with value: 0.42308950424194336 and parameters: {'layers': 4, 'nodes': 256, 'epochs': 10, 'learning_rate': 0.00028877658944409876, 'dropout_rate': 0.4655588859112405, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 8.93246828369214e-05}. Best is trial 0 with value: 0.42308950424194336.


Best trial: 0. Best value: 0.42309:   4%|▍         | 2/50 [01:02<27:10, 33.96s/it]

[I 2025-01-25 15:16:11,044] Trial 1 finished with value: 0.5178495049476624 and parameters: {'layers': 5, 'nodes': 128, 'epochs': 30, 'learning_rate': 0.0001497447529996826, 'dropout_rate': 0.1624125345662708, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 0.0008536127865776314}. Best is trial 0 with value: 0.42308950424194336.


Best trial: 0. Best value: 0.42309:   6%|▌         | 3/50 [01:18<20:10, 25.75s/it]

...

[I 2025-01-25 15:53:23,423] Trial 47 finished with value: 0.4379606544971466 and parameters: {'layers': 1, 'nodes': 128, 'epochs': 30, 'learning_rate': 0.0007978633535403383, 'dropout_rate': 0.31638567333101364, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 0.00021673587066476353}. Best is trial 46 with value: 0.3487260043621063.


Best trial: 46. Best value: 0.348726:  98%|█████████▊| 49/50 [38:48<00:26, 26.36s/it]

[I 2025-01-25 15:53:57,041] Trial 48 finished with value: 0.3507152497768402 and parameters: {'layers': 1, 'nodes': 256, 'epochs': 40, 'learning_rate': 0.0009484234359620656, 'dropout_rate': 0.40005432261502183, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 0.0006108099013293767}. Best is trial 46 with value: 0.3487260043621063.


Best trial: 46. Best value: 0.348726: 100%|██████████| 50/50 [39:22<00:00, 47.26s/it]

[I 2025-01-25 15:54:31,501] Trial 49 finished with value: 0.35163548588752747 and parameters: {'layers': 1, 'nodes': 256, 'epochs': 40, 'learning_rate': 0.0010360889572811538, 'dropout_rate': 0.40703317734843525, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 0.0005945091105117168}. Best is trial 46 with value: 0.3487260043621063.


In [22]:
# Best model training and saving
best_params = study.best_params
best_model = SalaryPredictor(
    X_train.shape[1], 
    best_params['nodes'], 
    best_params['layers'], 
    best_params['dropout_rate']
)

In [23]:
print("Best hyperparameters:", best_params)
print("Best validation loss:", study.best_value)

Best hyperparameters: {'layers': 1, 'nodes': 128, 'epochs': 40, 'learning_rate': 0.0009715619067182132, 'dropout_rate': 0.38247415095870707, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 0.00018542078372695}
Best validation loss: 0.3487260043621063


In [24]:
# Save the best model state
torch.save(best_model.state_dict(), 'Best_ANN_classification_model.h5')
# It allow to Preserve the model's learned parameters for future use 
# without having to retrain the model.

In [25]:
# Save the best hyperparameters in a pickle file
with open('Best_Hyperparameters.pkl', 'wb') as f:
    pickle.dump(best_params, f)

In [26]:
# Save the study object
with open('study.pkl', 'wb') as f:
    pickle.dump(study, f)

In [28]:
# Save the best model in onnx format
# dummy_input = torch.randn(1, X_train.shape[1])
# torch.onnx.export(best_model, dummy_input, 'Best_ANN_classification_model.onnx', verbose=True)

# Save the best model in torchscript format
# traced_model = torch.jit.trace(best_model, dummy_input)
# traced_model.save('Best_ANN_classification_model.pt')